<a href="https://colab.research.google.com/github/OswaldoMoper/Mathematical_Data_Analysis/blob/master/Data%20Analysis/Analysis_1_(SVM%2C_DTR%2C_RF)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [ ]:
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns = california.feature_names)
df['target'] = california.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
df.loc [ df.target > 4, 'famoso' ] = 1
df[ 'famoso' ] = df.famoso.fillna(0)
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target,famoso
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526,1.0
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585,0.0
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521,0.0
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413,0.0
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422,0.0
...,...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781,0.0
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771,0.0
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923,0.0
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847,0.0


In [ ]:
df.nunique()

MedInc        12928
HouseAge         52
AveRooms      19392
AveBedrms     14233
Population     3888
AveOccup      18841
Latitude        862
Longitude       844
target         3842
dtype: int64

In [ ]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [ ]:
fig = px.scatter_mapbox( df, lat = 'Latitude', lon = 'Longitude', color = 'target', hover_data = ['AveRooms', 'AveBedrms']
                       , mapbox_style = 'open-street-map', zoom = 3.8, title = 'Casas de California - Precio')
fig.update_layout(autosize = True)
fig.show()

In [ ]:
figh = px.histogram( df, x = 'target', nbins = 30, title = 'Distribución - Precios de Casas' )
figh.update_layout( xaxis_title = 'Precio', template = 'plotly_dark', bargap = 0.2)
figh.show()

In [ ]:
corr_matrix = df.corr()
figc = px.imshow( corr_matrix, x = corr_matrix.columns, y = corr_matrix.columns, color_continuous_scale = 'RdBu', range_color = [-1, 1],
                  text_auto = True, aspect = 'auto', title = 'Matriz de Correlación' )
figc.update( layout_coloraxis_showscale = False )
figc.show()

In [ ]:
figc2 = px.scatter_matrix( df, dimensions = df.columns[:-1], title = 'Pares de Variables', height = 1000, color = 'target')
figc2.show()

In [ ]:
X =  df.drop ('target', axis = 1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, random_state = 50 )

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
svm = SVR ()
svm.fit(X_train_scaled, y_train)

SVR()

In [ ]:
y_pred_svm = svm.predict(X_test_scaled)
mse_svm = mean_squared_error( y_test, y_pred = y_pred_svm )
print(mse_svm)

0.33746390576060625


In [ ]:
svm_r2 = r2_score( y_test, y_pred_svm )
print(svm_r2)

0.7402333105552517


In [ ]:
# param_grid_svm = {
#     'C':[1.0, 4, 2.718, 882, 0.01],
#     'kernel': [ 'linear', 'poly', 'rbf', 'sigmoid' ]
# }

In [ ]:
# svm2 = SVR()
# grid_search_svm = GridSearchCV( estimator = svm2, param_grid = param_grid_svm, cv = 5 )
# grid_search_svm.fit(X_train_scaled, y_train)

In [ ]:
# best_param_svm = grid_search_svm.best_estimator_
# best_param_svm

In [ ]:
dt = DecisionTreeRegressor( random_state = 50 )
dt.fit( X_train_scaled, y_train )

DecisionTreeRegressor(random_state=50)

In [ ]:
plt.figure ( figsize = (20,20), dpi = 200 )
plot_tree ( dt, feature_names = df.columns[:-1], filled = True )

[Text(0.6342762269463016, 0.9852941176470589, 'MedInc <= 0.618\nsquared_error = 1.34\nsamples = 16512\nvalue = 2.066'),
 Text(0.342840869227837, 0.9558823529411765, 'MedInc <= -0.416\nsquared_error = 0.838\nsamples = 13072\nvalue = 1.734'),
 Text(0.12893679434657734, 0.9264705882352942, 'AveOccup <= -0.083\nsquared_error = 0.553\nsamples = 6286\nvalue = 1.351'),
 Text(0.0374382568627374, 0.8970588235294118, 'Latitude <= 1.08\nsquared_error = 1.045\nsamples = 946\nvalue = 1.874'),
 Text(0.0217816327404381, 0.8676470588235294, 'Longitude <= 0.66\nsquared_error = 1.133\nsamples = 724\nvalue = 2.07'),
 Text(0.007862303488033729, 0.8382352941176471, 'Latitude <= -0.469\nsquared_error = 1.257\nsamples = 403\nvalue = 2.43'),
 Text(0.002884714942638159, 0.8088235294117647, 'MedInc <= -0.692\nsquared_error = 1.025\nsamples = 151\nvalue = 2.987'),
 Text(0.000887208252999603, 0.7794117647058824, 'AveOccup <= -0.157\nsquared_error = 0.907\nsamples = 71\nvalue = 2.663'),
 Text(0.0002512448150087371

Error in callback <function _draw_all_if_interactive at 0x78d68cf11120> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x78d68cf103a0> (for post_execute):


KeyboardInterrupt: 

In [ ]:
plt.show()

In [ ]:
y_pred_dt = dt.predict( X_test_scaled )
mse_dt = mean_squared_error( y_test, y_pred_dt )
print( mse_dt )

0.6241750131550386


In [ ]:
dt_r2 = r2_score( y_test, y_pred_dt )
print( dt_r2 )

0.5195341663696705


In [ ]:
rf = RandomForestRegressor ( random_state = 50 )
rf.fit ( X_train_scaled, y_train )

RandomForestRegressor(random_state=50)

In [ ]:
y_pred_rf = rf.predict ( X_test_scaled )
mse_rf =  mean_squared_error ( y_test, y_pred_rf )
print ( mse_rf )

0.30930362678769885


In [ ]:
rf_r2 = r2_score ( y_test, y_pred_rf )
print(rf_r2)

0.7619100063966789


In [ ]:
results =  pd.DataFrame({
    'Model': [ 'SVM', 'DT', 'RF' ],
    'MSE': [ mse_svm, mse_dt, mse_rf ],
    'R2': [ svm_r2, dt_r2, rf_r2 ]
 })
results

,Model,MSE,R2
0,SVM,0.337464,0.740233
1,DT,0.624175,0.519534
2,RF,0.309304,0.761910


In [ ]:
figr = px.bar(results, x = 'Model', y = [ 'MSE', 'R2' ], barmode = 'group', title = 'Comparación Modelos',
              labels = { 'value': 'Score', 'variable': 'Metrica' }, template = 'plotly_dark' )
figr.show()